In [42]:
import os 
import keras
import numpy as np 
from tqdm import tqdm 
import tensorflow as tf

import pandas as  pd
# Data 
import scipy.ndimage as mp
from glob import glob
import pathlib
import cv2 as cv
from keras.preprocessing.image import img_to_array, load_img
from tensorflow.image import resize
import tensorflow.image as tfi


# Data visualization
import matplotlib.pyplot as plt


from PIL import Image

import subprocess
from IPython.display import FileLink, display

from keras.utils import  Sequence

from tqdm import tqdm, trange
import shutil
#from tqdm.notebook import tqdm didnt work properly on kaggel
#from tqdm.auto import tqdm
#from time import sleep


In [47]:
images = glob(r'../input/aisegmentcom-matting-human-datasets/clip_img/*/*/*')
masks = glob(r'../input/aisegmentcom-matting-human-datasets/matting/*/*/*')

In [48]:
len(images)

34426

# Preprocessing 

## Dell mask and images
dell data where mask take area more than 0.85 and less than 0.25 of original image 

In [49]:

def del_mask_img(images, masks):
    for im, ma in tqdm(zip(images, masks), total = len(images)):
        #sleep(0.00001)
        mask = load_img(ma) 
        mask= img_to_array(mask)[:,:,0] #np
        mask = np.where(mask > 0, 1, 0)
        m_sum = np.sum(mask) 
        mask_to_img_relation = m_sum/(mask.shape[0]*mask.shape[1])
        if mask_to_img_relation >= 0.85 or mask_to_img_relation <= 0.25:
            masks.remove(ma)
            images.remove(im)
    return images, masks


In [50]:
images, masks = del_mask_img(images, masks)

 98%|█████████▊| 33653/34426 [21:26<00:29, 26.16it/s]


## Fill masks

In [51]:

def fill_mask(mask):
    # See for mask as vector string and fill with one this string from first non zero value to the last non zero value
    m = plt.imread(mask)
    m = m[:,:,0]
    m = (m[:,:]>0).astype(np.float16)
    
    for i, vector in enumerate(m):
        Z = np.zeros_like(vector)
        O = np.ones_like(vector)
        one_indexs = []
        
        for i2, el in enumerate(vector):
            if el > 0:
                one_indexs.append(i2)
        if one_indexs:
            Z[one_indexs[0]:one_indexs[-1]] = O[one_indexs[0]:one_indexs[-1]]
            m[i,:] = Z
    return m
    
    

## Copy masks

In [ ]:

def copy_masks(masks):
    new_dir = os.path.join('/kaggle/working/', 'masks')
    try:
        os.mkdir(new_dir)    
    except FileExistsError:
        pass

    for masks_path in tqdm(masks, total = len(masks)):
        mask = fill_mask(masks_path)
        path = masks_path.split('/')
        new_dir = os.path.join('/kaggle/working/masks',path[-3], path[-2])
        try:
            os.makedirs(new_dir) 
        except FileExistsError:
            pass
        
        np.save(os.path.join(new_dir, path[-1][:-4]), mask) #path[-1] 



        

(masks)

import shutil
shutil.make_archive('masks', 'zip', '/kaggle/working/masks/')